# Lab 7a: on Colab

TODO:
- Click on the link, open with Colab, and save it as your own copy: <a href="https://drive.google.com/file/d/1YK8cMhgjX6CXQ3GX3_04nynJlBO70ncO/view?usp=sharing"> Colab Link</a>
- After finishing it, download the ipynb and upload it to canvas as before.


Why we switch:
- BC's Jupyterhub doesn't support GPU, plus some library configuration doesn't work out for pytorch... We'll use Colab from now on for the deep learning technology.
- Colab is the future for cloud computing. There are research labs develop "zero-cost" deep learning framework by exploiting Google's cloud infrastructure: <a href="https://github.com/HenriquesLab/ZeroCostDL4Mic"> link </a>


